In [1]:
import geopandas as gpd
import os
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pickle
import copy 

warnings.filterwarnings('ignore')
tqdm.pandas()

# Construct Roads Graph

## Load Data

In [2]:
MAPA_PATH = "../../../data/Mapa/Ben Gurion University/Gisrael/"
roads_df = gpd.read_file(MAPA_PATH + 'is_str.shp')

## Basic Edges

In [117]:
shape_edges_df = roads_df[['UserID', 'ObjID', 'FJunction', 'TJunction', 'RoadType', 
                     'OneWay', 'FromSpeedL', 'ToSpeedL', 
                     'AprxSpeedL', 'Seconds', 'Length', 'Roaddirect', 'DirectionM', 'F_ZLev', 'T_ZLev']]
shape_edges_df = shape_edges_df.reset_index()

In [158]:
edges_dict = dict()
adjacency_list = dict()
number_of_two_way = 0
number_of_ft = 0
number_of_tf = 0

def create_edges_dict(e):
    global edges_dict
    global number_of_two_way
    global number_of_ft
    global number_of_tf
    
    road_type = e['OneWay']
    frm = str(e['FJunction'])
    to = str(e['TJunction'])
    
    if road_type == 'N':
        return
    elif road_type == 'ft':
        if (frm, to) in edges_dict:
            raise Exception(f'Found parallel edge: {frm}, {to}')
        edges_dict[frm, to] = copy.deepcopy(e)
        
        if frm not in adjacency_list:
            adjacency_list[frm] = dict()
        adjacency_list[frm][to] = copy.deepcopy(e)
        
        number_of_ft += 1
    elif road_type == 'tf':
        if (to, frm) in edges_dict:
            raise Exception(f'Found parallel edge: {to}, {frm}')
        edges_dict[to, frm] = copy.deepcopy(e)
        
        if to not in adjacency_list:
            adjacency_list[to] = dict()
        adjacency_list[to][frm] = copy.deepcopy(e)
        
        number_of_tf += 1
    elif road_type == '2':
        if (frm, to) in edges_dict:
            raise Exception(f'Found parallel edge: {frm}, {to}')
        if (to, frm) in edges_dict:
            raise Exception(f'Found parallel edge: {to}, {frm}')
        edges_dict[frm, to] = copy.deepcopy(e)
        edges_dict[to, frm] = copy.deepcopy(e)
        
        if frm not in adjacency_list:
            adjacency_list[frm] = dict()
        adjacency_list[frm][to] = copy.deepcopy(e)
        
        if to not in adjacency_list:
            adjacency_list[to] = dict()
        adjacency_list[to][frm] = copy.deepcopy(e)
        
        number_of_two_way +=2
        
res = shape_edges_df.progress_apply(lambda x: create_edges_dict(x), axis=1)

In [160]:
original_adjacency_list = adjacency_list.copy()

### Test

In [161]:
# Sanity check
number_of_edges = len(edges_dict)
number_of_ft = shape_edges_df[shape_edges_df['OneWay'] == 'ft'].shape[0]
number_of_tf = shape_edges_df[shape_edges_df['OneWay'] == 'tf'].shape[0]
number_of_two_way = shape_edges_df[shape_edges_df['OneWay'] == '2'].shape[0]
print(number_of_edges)
print(f'number of ft: {number_of_ft}, number of tf: {number_of_tf}, number of two ways * 2: {number_of_two_way*2}')
assert number_of_edges == (number_of_ft + number_of_tf + number_of_two_way*2)

859377
number of ft: 154969, number of tf: 24314, number of two ways * 2: 680094


## Turn Restrictions

In [162]:
turn_restrict_df = gpd.read_file(MAPA_PATH + 'restrict.shp')

### Utils

In [163]:
def get_edge_other_node(edge_id, node):
    e = shape_edges_df[shape_edges_df['UserID'] == edge_id]
    
    f = e['FJunction'].values[0]
    t = e['TJunction'].values[0]
    
    if f == node:
        return t
    if t == node:
        return f
    
    raise Exception(f'node {node} is not part of the edge {edge_id}')
    
    
############################################################    
######################## Test ##############################
############################################################

assert get_edge_other_node(1186054, 1058686) == 1058685

### Manipulate Graph according to [Turn Restrictions Algorithm](https://docs.google.com/document/d/1XG5cRIPrrrchI-ub2eU_BYekPyKPA4vKLt3xK62fkS8/edit?usp=sharing)

In [164]:
skipped_restrictions = []

for i, restrict in turn_restrict_df.iterrows():
    u = str(get_edge_other_node(restrict['einterval'], restrict['junctionid']))
    v = str(restrict['junctionid'])
    w = str(get_edge_other_node(restrict['xinterval'], restrict['junctionid']))
    
    original_v = v
    # maybe we already created v_u
    if (u in adjacency_list) and (v not in adjacency_list[u]):
        v = v + '_' + u
    # maybe we already created w_v
    if (v in adjacency_list) and (w not in adjacency_list[v]):
        w = w + '_' + original_v    
    
    # if no edge (u,v) or (v,w) then the restriction is already enforced
    if ((u not in adjacency_list) or (v not in adjacency_list[u]) or 
        (v not in adjacency_list) or (w not in adjacency_list[v])):
        skipped_restrictions.append(restrict)
        continue
        
    if '_' in v:  # v already represents v_u
        # remove (v_u,w)
        del adjacency_list[v][w]
    else:  # v_u was not already created
        # add a new node v_u with a new edge (u, v_u)
        v_u = v + '_' + u
        adjacency_list[v_u] = dict()
        adjacency_list[u][v_u] = adjacency_list[u][v]
    
        # remove (u,v)
        del adjacency_list[u][v]
    
    
        # for each edge (v,x) where x!=w add (v_u, x)
        for x in adjacency_list[v]:
            if x == w:
                continue
            adjacency_list[v_u][x] = adjacency_list[v][x]
        
print(f'Skipped {len(skipped_restrictions)} restrictions')

Skipped 1448 restrictions


In [165]:
# Verify that all skipped restrictions are blocked roads
count = 0
for row in skipped_restrictions:    
    f_edge_type = shape_edges_df[shape_edges_df['UserID'] == row['einterval']]['OneWay'].values[0] 
    t_edge_type = shape_edges_df[shape_edges_df['UserID'] == row['xinterval']]['OneWay'].values[0] 
    if f_edge_type != 'N' and t_edge_type != 'N':
        count += 1
count

0

In [166]:
# save adjacency list with turn restrictions
with open('../../output_data/roads_graph_turn_restrictions.pkl','wb') as f:
    pickle.dump(adjacency_list, f)

## Z-Levels
In order to handle z-levels, we must go through all pairs of (u,v),(v,w) edges, and compare T_ZLev(u,v) and F_ZLev(v,w). If they are different, we will mark u->v->w as a turn restriction. 

In [167]:
shape_edges_df.head(2)

,index,UserID,ObjID,FJunction,TJunction,RoadType,OneWay,FromSpeedL,ToSpeedL,AprxSpeedL,Seconds,Length,Roaddirect,DirectionM,F_ZLev,T_ZLev
0,0,1004561,1,947733,947736,254,2,10,10,10,69.734908,64.569179,222222,None,0,0
1,1,1059914,2,950536,983821,10,2,90,90,90,2.249966,49.999104,222222,B,0,0


In [168]:
edges = {'From': [], 'To': [], 'Edge': []}
for f in original_adjacency_list:
    for t in original_adjacency_list[f]:
        edges['From'].append(f)
        edges['To'].append(t)
        edges['Edge'].append(original_adjacency_list[f][t])

edges_df = pd.DataFrame(edges)
    

In [169]:
edges_df.head()

,From,To,Edge
0,947733,947736,index 0 UserID 1004561 Ob...
1,947733,947732,index 1542 UserID 1004557 Ob...
2,947736,947733,index 0 UserID 1004561 Ob...
3,947736,947735,index 1550 UserID 1004560 Ob...
4,950536,983821_950536,index 1 UserID 1059914 Ob...


In [170]:
pairs_of_edges_df = edges_df.merge(edges_df, left_on='To', right_on='From', suffixes=('_F', '_T'))

In [171]:
pairs_of_edges_df.head()

,From_F,To_F,Edge_F,From_T,To_T,Edge_T
0,947733,947736,index 0 UserID 1004561 Ob...,947736,947733,index 0 UserID 1004561 Ob...
1,947733,947736,index 0 UserID 1004561 Ob...,947736,947735,index 1550 UserID 1004560 Ob...
2,947735,947736,index 1550 UserID 1004560 Ob...,947736,947733,index 0 UserID 1004561 Ob...
3,947735,947736,index 1550 UserID 1004560 Ob...,947736,947735,index 1550 UserID 1004560 Ob...
4,947733,947732,index 1542 UserID 1004557 Ob...,947732,947735,index 1043 UserID 1004570 Ob...


In [172]:
edges_df.shape

(859377, 3)

In [173]:
pairs_of_edges_df.shape

(1850554, 6)

In [174]:
z_level_restrictions = {'einterval': [], 'junctionid': [], 'xinterval': []}
for _, row in pairs_of_edges_df.iterrows():    
    middle_node_is_tjunction_in_edge_f = row['To_F'] == str(row['Edge_F']['TJunction'])
    if middle_node_is_tjunction_in_edge_f:
        from_zlev = row['Edge_F']['T_ZLev']
    else:
        assert row['To_F'] == str(row['Edge_F']['FJunction']), f"Edge_F is drawn from {row['Edge_F']['FJunction']} to {row['Edge_F']['TJunction']}, \
                while the current edge goes to {row['To_F']}"
        from_zlev = row['Edge_F']['F_ZLev']
        
    middle_node_is_fjunction_in_edge_t = row['From_T'] == str(row['Edge_T']['FJunction'])
    if middle_node_is_fjunction_in_edge_t:
        to_zlev = row['Edge_T']['F_ZLev']
    else:
        assert row['From_T'] == str(row['Edge_T']['TJunction']), f"Edge_T is drawn from {row['Edge_T']['FJunction']} to {row['Edge_T']['TJunction']}, \
                while the current edge goes from {row['From_F']}"
        to_zlev = row['Edge_T']['T_ZLev']
        
    if from_zlev != to_zlev:
        z_level_restrictions['einterval'].append(int(row['Edge_F']['UserID']))
        z_level_restrictions['junctionid'].append(int(row['To_F']))
        z_level_restrictions['xinterval'].append(int(row['Edge_T']['UserID']))
        
        
z_level_restrictions_df = pd.DataFrame(z_level_restrictions)

In [175]:
z_level_restrictions_df.head(10)

,einterval,junctionid,xinterval
0,1614399,887616,1125452
1,1614399,887616,1125453
2,1125452,887616,12090
3,1125452,887616,1614399
4,12090,887616,1125452
5,12090,887616,1125453
6,1125453,887616,12090
7,1125453,887616,1614399
8,1391723,1181793,1391721
9,1391723,1181793,1391722


In [176]:
skipped_restrictions = []

for i, restrict in z_level_restrictions_df.iterrows():
    u = str(get_edge_other_node(restrict['einterval'], restrict['junctionid']))
    v = str(restrict['junctionid'])
    w = str(get_edge_other_node(restrict['xinterval'], restrict['junctionid']))
    
    original_v = v
    # maybe we already created v_u
    if (u in adjacency_list) and (v not in adjacency_list[u]):
        v = v + '_' + u
    # maybe we already created w_v
    if (v in adjacency_list) and (w not in adjacency_list[v]):
        w = w + '_' + original_v    
    
    # if no edge (u,v) or (v,w) then the restriction is already enforced
    if ((u not in adjacency_list) or (v not in adjacency_list[u]) or 
        (v not in adjacency_list) or (w not in adjacency_list[v])):
        skipped_restrictions.append(restrict)
        continue
        
    if '_' in v:  # v already represents v_u
        # remove (v_u,w)
        del adjacency_list[v][w]
    else:  # v_u was not already created
        # add a new node v_u with a new edge (u, v_u)
        v_u = v + '_' + u
        adjacency_list[v_u] = dict()
        adjacency_list[u][v_u] = adjacency_list[u][v]
    
        # remove (u,v)
        del adjacency_list[u][v]
    
    
        # for each edge (v,x) where x!=w add (v_u, x)
        for x in adjacency_list[v]:
            if x == w:
                continue
            adjacency_list[v_u][x] = adjacency_list[v][x]
        
print(f'Skipped {len(skipped_restrictions)} restrictions')

Skipped 0 restrictions


In [177]:
# save adjacency list with turn restrictions
with open('../../output_data/roads_graph_turn_restrictions_and_z_levels.pkl','wb') as f:
    pickle.dump(adjacency_list, f)

## Add Weights

In [185]:
weighted_edges = []

for f in adjacency_list:
    for t in adjacency_list[f]:
        weighted_edges.append((f,t,adjacency_list[f][t]['Seconds']))

In [186]:
len(weighted_edges)

949378

In [187]:
with open('../../output_data/roads_graph_weighted_edges.pkl','wb') as f:
    pickle.dump(weighted_edges, f)